## Signal to Noise for Q-MIC with gated MPD SPC3 @ 800 nm

Shows coincidence to accidentals ratio (CAR) and total signal to noise ratio (SNR) for
- varying gate windows,
- varying dark counts per pixel,
- different detection efficiencies (total detection efficiency incl. transmission and quantum efficiency): with and without microlens array,
- full sensor vs partial use of sensor.

Also displays corresponding valid pair rate as a function of the above

In [2]:
# Initialise system values
#-------------------------

import trace as tr
import numpy as np

# target SPDC pair generation rate is 10MHz
pairRate = 1E7

quantEff = 0.05 # SPC3 quantum efficiency at 800nm

optTrans = 0.5 # 50% optical losses
fillFactor = np.array([(30.0**2)/(150.0**2),0.7]) # without and with microlens array (70% fill factor claimed with MLA)
nf = fillFactor.size # number of fillFactor elements
eff = quantEff*optTrans*fillFactor

darkC_ppix = 100 # for >80% of the SPAD pixels dark count is <100cps
numpix = np.array([512,2048]) # focusing on 1/4 of array or using full array
nc = numpix.size # number of pixel configurations

frameRate = 96E3
frameT = 1/frameRate *1E9 # frame period in nanoseconds

ng = 17 # number of gateWin elements
gateWin = np.linspace(1.0,5.0,ng) # different gate windows in nanoseconds
dutyCyc = gateWin/frameT # duty cycles for each gate window (time gate ON vs total length of frame)

pairs_psec = pairRate * eff**2 # number of SPDC photon pairs detected by SPAD array per second before gating
darkCSingles_psec = darkC_ppix * numpix # number of dark count singles detected by SPAD array per second before gating
accSingles_psec = pairRate *(eff-eff**2) # number of accidental singles detected by SPAD array per second before gating

print(pairs_psec.size, darkCSingles_psec.size,accSingles_psec.size)

(2, 2, 2)


In [15]:
# Calculate CAR
#--------------

# Apply gating and calculate probabilities (per frame) of coincidences and accidentals
# (need to always have probability of detection << 1 per frame):
# CAR varies with gateWin and with fillFactor , therefore is ng*nf 2D-array
prob_realCoinc = np.tile(gateWin.reshape(ng,1), [1,nf]) * np.tile(pairs_psec, [ng,1]) #prob real SPDC coincidence
prob_accSingle = np.tile(gateWin.reshape(ng,1), [1,nf]) * np.tile(accSingles_psec, [ng,1]) #prob accidental SPDC single photon
prob_accCoinc = prob_accSingle**2 #prob accidental coincidence (from two unpaired SPDC photons)

CAR = prob_realCoinc/prob_accCoinc
CAR.shape

(17, 2)

In [ ]:
# Calculate full SNR
#-------------------

# Apply gating and calculate probabilities (per frame) for dark counts too:
# darkC varies with numpix, therefore prob_darkC are 2D arrays with size ng*npix
 prob_darkCSingle = dutyCycle*darkCSingles_psec/frameRate;
    prob_triple = prob_darkCSingle*prob_realCoinc;
    prob_darkAccCoinc = 2* prob_darkCSingle*prob_accSingle;
    
    prob_validPair = prob_realCoinc - prob_triple;
    prob_noiseCoinc = prob_accCoinc + prob_darkAccCoinc;
    
    SNR(gateCounter) = prob_validPair/prob_noiseCoinc;
    
    validPairRate(gateCounter) = prob_validPair * frameRate;